In [1]:
#this notebook aims to understand the LSS contribution
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [20]:

def data_in_halobin(m0, m1, ljack):
    # getting the data
    dat = pd.read_csv('halo_catalog.csv')
    dat['x'] = dat['x']%2500
    dat['y'] = dat['y']%2500
    dat['z'] = dat['z']%2500
    #assigning jackknife region
    ljacks = ljack
    jlen = 2500/ljacks
    ix = dat['x']//jlen; iy = dat['y']//jlen; iz = dat['z']//jlen
  
    xjkreg = ix + iy*ljacks + iz*ljacks**2
    njacks = len(np.unique(xjkreg))
    print('njacks=%d'%len(np.unique(xjkreg)))
    idx = (np.log10(dat['Mvir'])>m0) & (np.log10(dat['Mvir'])<m1)
    return dat[idx], xjkreg[idx]


In [33]:
def get_xi(logm0, logm1, ljack):
    dat,xjkreg = data_in_halobin(logm0, logm1, ljack)
    njacks = len(np.unique(xjkreg))

    nhalo = len(xjkreg)/(2500)**3
    rbins = np.logspace(np.log10(4), np.log10(20), 11)
    rad   = 0.5*(rbins[:-1] + rbins[1:])

    RR = 4*np.pi*(0.5*(rbins[:-1] + rbins[1:]))**2 * (rbins[1:] - rbins[:-1]) * nhalo
    

    from scipy.spatial import cKDTree
    xi = np.zeros(len(rad)*njacks)
    rrad = 0.0 *xi

    # tree over full halo catalog
    htree = cKDTree(np.transpose([dat['x'], dat['y'], dat['z']]), boxsize=2500)

    # intrbins, x0 = jj*len(ra), x1 = (jj+1)* len(ra)

    for jj in range(njacks):
        idx  = (xjkreg!=jj)
    
        # tree over jackknifed catalog
        jtree = cKDTree(np.transpose([dat['x'][idx], dat['y'][idx], dat['z'][idx]]), boxsize=2500)
        DD = htree.count_neighbors(jtree, rbins, cumulative=True)
        DD = np.diff(DD)/sum(idx)
        x0 = jj*len(DD); x1 = (jj+1)* len(DD)
    
        xi[x0:x1] = DD/RR - 1
        rrad[x0:x1] = rad
        print(jj, DD/RR - 1)
    #np.loadtxt('./xi_%s_%s.dat'%(logm0,logm1), np.transpose([rrad,xi]))
    return rrad, xi

In [34]:
logM0=12
logM1=13

xx,yy = get_xi(logM0, logM1, ljack=2)


njacks=8
0 [1.03227676 0.79242632 0.66303675 0.52245611 0.38140118 0.30091917
 0.21086276 0.16673377 0.12897392 0.08695607]
1 [1.06490841 0.8059424  0.6643793  0.51935927 0.38103452 0.3125861
 0.21562801 0.17093303 0.13007445 0.08848015]
2 [1.04148839 0.80825092 0.64468537 0.51785198 0.38161906 0.30293216
 0.2207324  0.16936113 0.12924955 0.08804793]
3 [1.04923524 0.8096786  0.66009041 0.52969837 0.38785186 0.30530484
 0.21144439 0.16544212 0.12687887 0.08588504]
4 [1.05548371 0.81265199 0.6590651  0.52404974 0.3718584  0.29913497
 0.21390174 0.16546079 0.12635479 0.08926785]
5 [1.05662703 0.82080144 0.66754521 0.5195385  0.3848954  0.29990887
 0.21505286 0.16747529 0.12999758 0.08889529]
6 [1.03112381 0.79987781 0.66587833 0.52552236 0.37681159 0.30789146
 0.22375326 0.16514821 0.12981588 0.0884427 ]
7 [1.03666637 0.8251957  0.65309658 0.52082367 0.38799887 0.30254632
 0.21886133 0.17103357 0.12825662 0.09051024]


TypeError: 'int' object is not subscriptable

In [28]:
rrad

array([ 4.34923789,  5.10869721,  6.00077252,  7.04862107,  8.27944383,
        9.72519156, 11.42339424, 13.41813526, 15.76119586, 18.51339923,
        4.34923789,  5.10869721,  6.00077252,  7.04862107,  8.27944383,
        9.72519156, 11.42339424, 13.41813526, 15.76119586, 18.51339923,
        4.34923789,  5.10869721,  6.00077252,  7.04862107,  8.27944383,
        9.72519156, 11.42339424, 13.41813526, 15.76119586, 18.51339923,
        4.34923789,  5.10869721,  6.00077252,  7.04862107,  8.27944383,
        9.72519156, 11.42339424, 13.41813526, 15.76119586, 18.51339923,
        4.34923789,  5.10869721,  6.00077252,  7.04862107,  8.27944383,
        9.72519156, 11.42339424, 13.41813526, 15.76119586, 18.51339923,
        4.34923789,  5.10869721,  6.00077252,  7.04862107,  8.27944383,
        9.72519156, 11.42339424, 13.41813526, 15.76119586, 18.51339923,
        4.34923789,  5.10869721,  6.00077252,  7.04862107,  8.27944383,
        9.72519156, 11.42339424, 13.41813526, 15.76119586, 18.51

In [24]:

logM0=12
logM1=13

rrad, xi = get_xi(logM0, logM1, ljack=5)
njacks = ljack**3



from colossus.cosmology import cosmology
cosmo = cosmology.setCosmology('planck18')
ximm = cosmo.correlationFunction(rad,0.0)
bias = np.sqrt(xi/np.tile(ximm, njacks))
#saving the bias for logMh 11 threshold
bias11 = bias.reshape((njacks, len(rad)))
print(np.mean(rrad.reshape((njacks, len(rad))),  axis=0))

yy = np.mean(bias11, axis=0)
print(yy)
cov     = np.zeros((len(rad),len(rad)))
print(njacks)
for ii in range(len(rad)):
    for jj in range(len(rad)):
  
        cov[ii][jj] = np.mean((bias11[ii,:] - yy[ii])*(bias11[jj,:] - yy[jj]))
        cov[ii][jj] = (njacks - 1)*cov[ii][jj]



yyerr = np.sqrt(np.diag(cov))

plt.subplot(2,2,1)
plt.errorbar(rad, yy, yerr=yyerr, fmt='.', capsize=3)
plt.ylim(0.65,1.1)
plt.xlabel(r'$r \, [{\rm h^{-1}Mpc}]$')
plt.ylabel(r'$b_{h}$')
plt.xscale('log')    
plt.title(r'$%s<\log[M_{\rm vir}/({\rm h^{-1}M_{\odot}})] < %s$'%(logM0, logM1))


corr = 0.0*cov
for ii in range(len(rad)):
    for jj in range(len(rad)):
        corr[ii][jj] = cov[ii][jj]*1.0/(yyerr[ii]*yyerr[jj])


plt.subplot(2,2,2)
plt.imshow(corr,cmap='PuOr_r',vmin=-1,vmax=1,origin='lower',aspect='equal')
plt.xlabel(r'$b_{\rm h}$')
plt.colorbar()



plt.subplot(2,2,3)
for jjx in np.random.randint(njacks, size=10):
    x0 = jjx*len(rad); x1 = (jjx+1)* len(rad)
    plt.plot(rad,xi[x0:x1])
plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'$r \, [{\rm h^{-1}Mpc}]$')
plt.ylabel(r'$\xi(r)$')
plt.ylim(0.05,2)
plt.tight_layout()

plt.savefig('logMh_%s_%s.png'%(logM0,logM1))

njacks=125
0 [1.04250543 0.80953971 0.66039686 0.52447624 0.38337852 0.30423956
 0.21647607 0.16781147 0.12910611 0.08850905]
1 [1.04511113 0.80614542 0.66189462 0.52090685 0.38186185 0.30357605
 0.21581787 0.16729817 0.12866233 0.08780992]
2 [1.04295383 0.81025495 0.65710684 0.5223979  0.38054593 0.30354936
 0.21525596 0.16811507 0.12897803 0.08804278]
3 [1.04969528 0.81111487 0.65733993 0.52187941 0.38052215 0.30499836
 0.21651452 0.16826094 0.12933041 0.08846984]
4 [1.05253503 0.80966889 0.66114257 0.52141175 0.38123269 0.30409692
 0.21629017 0.1683997  0.12850137 0.08839026]
5 [1.04399519 0.80838407 0.65901483 0.52397191 0.38193499 0.30487571
 0.21638239 0.16813659 0.1293514  0.08876263]
6 [1.04476628 0.8118243  0.66175669 0.52321751 0.3836186  0.30397883
 0.215736   0.16861336 0.12887539 0.08769495]
7 [1.04715215 0.80916375 0.65917065 0.5246568  0.38311538 0.30358389
 0.21656936 0.16777512 0.12877908 0.08846765]
8 [1.04485463 0.8102761  0.66170305 0.52243186 0.38229328 0.3046246
 

71 [1.04185039 0.80994966 0.6614034  0.52014512 0.38233054 0.30421255
 0.21738742 0.16738736 0.1296274  0.08860079]
72 [1.04219517 0.81036981 0.65828019 0.52162701 0.3832005  0.30224813
 0.21656022 0.16825996 0.12903365 0.08883108]
73 [1.04459727 0.81126797 0.6607413  0.52254983 0.38340873 0.30440146
 0.2158537  0.16823629 0.12886515 0.08797239]
74 [1.04860075 0.81476813 0.65927662 0.5236706  0.38172324 0.30337247
 0.21650161 0.16773751 0.12871965 0.08804595]
75 [1.04697267 0.81214238 0.66180096 0.52337531 0.38202276 0.30562431
 0.21520386 0.16799531 0.12870256 0.08830819]
76 [1.04634563 0.81022584 0.66096533 0.52201869 0.37959193 0.30225913
 0.2155276  0.16629394 0.12834799 0.0880988 ]
77 [1.04507562 0.80927866 0.6612218  0.52318932 0.38271357 0.30440417
 0.21652813 0.1673591  0.12913235 0.0886067 ]
78 [1.04949894 0.81155113 0.657432   0.52041812 0.38229983 0.30248694
 0.21667068 0.16791348 0.12898691 0.08888233]
79 [1.04735468 0.81088099 0.66157622 0.52162151 0.38197912 0.30312005
 0

TypeError: cannot unpack non-iterable int object

In [ ]:
corr = 0.0*cov
for ii in range(len(rad)):
    for jj in range(len(rad)):
        corr[ii][jj] = cov[ii][jj]*1.0/(yyerr[ii]*yyerr[jj])


plt.subplot(2,2,1)
plt.imshow(corr,cmap='PuOr_r',vmin=-1,vmax=1,origin='lower',aspect='equal')
plt.colorbar()

In [ ]:
plt.subplot(2,2,1)
for jjx in np.random.randint(njacks, size=10):
    x0 = jjx*len(rad); x1 = (jjx+1)* len(rad)
    plt.plot(rad,xi[x0:x1])
plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'$r \, [{\rm h^{-1}Mpc}]$')
plt.ylabel(r'$\xi(r)$')
